In [0]:
import os
import sys
sys.path.append(os.getcwd())

from openrec import ImplicitModelTrainer
from openrec.utils import ImplicitDataset
from openrec.recommenders import ConcatVisualBPR
from openrec.utils.evaluators import AUC
from openrec.utils.samplers import PairwiseSampler

In [0]:
import tensorflow as tf

sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess_config.gpu_options.visible_device_list = '1'

In [2]:
!ls drive/start_up/

ctrsr_datasets	dataset


In [5]:
import os
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

urlretrieve('http://www.wanghao.in/data/ctrsr_datasets.rar', 'ctrsr_datasets.rar')
os.system('unrar x ctrsr_datasets.rar')

32512

In [0]:
import numpy as np
import random

users_count = 0
interactions_count = 0
with open('drive/start_up/ctrsr_datasets/citeulike-a/users.dat', 'r') as fin:
    for line in fin:
        interactions_count += int(line.split()[0])
        users_count += 1

# radomly hold out an item per user for validation and testing respectively.
val_structured_arr = np.zeros(users_count, dtype=[('user_id', np.int32), ('item_id', np.int32)]) 
test_structured_arr = np.zeros(users_count, dtype=[('user_id', np.int32), ('item_id', np.int32)])
train_structured_arr = np.zeros(interactions_count-11102, dtype=[('user_id', np.int32), ('item_id', np.int32)])

interaction_ind = 0
next_user_id = 0
next_item_id = 0
map_to_item_id = dict()  # Map item id from 0 to len(items)-1

with open('drive/start_up/ctrsr_datasets/citeulike-a/users.dat', 'r') as fin:
    for line in fin:
        item_list = line.split()[1:]
        random.shuffle(item_list)
        for ind, item in enumerate(item_list):
            if item not in map_to_item_id:
                map_to_item_id[item] = next_item_id
                next_item_id += 1
            if ind == 0:
                val_structured_arr[next_user_id] = (next_user_id, map_to_item_id[item])
            elif ind == 1:
                test_structured_arr[next_user_id] = (next_user_id, map_to_item_id[item])
            else:
                train_structured_arr[interaction_ind] = (next_user_id, map_to_item_id[item])
                interaction_ind += 1
        next_user_id += 1

In [0]:
from openrec.utils import ImplicitDataset

train_dataset = ImplicitDataset(raw_data=train_structured_arr, 
                        max_user=users_count, 
                        max_item=len(map_to_item_id), name='Train')
val_dataset = ImplicitDataset(raw_data=val_structured_arr, 
                      max_user=users_count,
                      max_item=len(map_to_item_id), name='Val')
test_dataset = ImplicitDataset(raw_data=test_structured_arr, 
                       max_user=users_count,
                       max_item=len(map_to_item_id), name='Test')

In [0]:
batch_size = 1000
test_batch_size = 100
display_itr = 10000

In [15]:
from openrec.recommenders import CML

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
                dim_embed=20, sess_config=sess_config)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [0]:
from openrec.utils.samplers import PairwiseSampler
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=5)


In [0]:
from openrec import ImplicitModelTrainer
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size, 
    train_dataset=train_dataset, model=cml_model, sampler=sampler)




In [0]:
from openrec.utils.evaluators import AUC
auc_evaluator = AUC()


In [0]:
11:25

In [19]:

model_trainer.train(num_itr=int(1e5), display_itr=display_itr, eval_datasets=[val_dataset, test_dataset],
                    evaluators=[auc_evaluator])

== Start training with FULL evaluation ==
[Itr 1000] Finished
[Itr 2000] Finished
[Itr 3000] Finished
[Itr 4000] Finished
[Itr 5000] Finished
[Itr 6000] Finished
[Itr 7000] Finished
[Itr 8000] Finished
[Itr 9000] Finished
[Itr 10000] Finished
[Itr 10000] loss: 499.617189
..(dataset: Val) evaluation


100%|██████████| 56/56 [01:08<00:00,  1.23s/it]

..(dataset: Val) AUC 0.9126019580364682
..(dataset: Test) evaluation



100%|██████████| 56/56 [01:10<00:00,  1.26s/it]


..(dataset: Test) AUC 0.91129645985358
[Itr 11000] Finished
[Itr 12000] Finished
[Itr 13000] Finished
[Itr 14000] Finished
[Itr 15000] Finished
[Itr 16000] Finished
[Itr 17000] Finished
[Itr 18000] Finished
[Itr 19000] Finished
[Itr 20000] Finished
[Itr 20000] loss: 101.435268
..(dataset: Val) evaluation


...

..(dataset: Test) AUC 0.952355147305938
[Itr 81000] Finished
[Itr 82000] Finished
[Itr 83000] Finished
[Itr 84000] Finished
[Itr 85000] Finished
[Itr 86000] Finished
[Itr 87000] Finished
[Itr 88000] Finished
[Itr 89000] Finished
[Itr 90000] Finished
[Itr 90000] loss: 57.228166
..(dataset: Val) evaluation


100%|██████████| 56/56 [00:56<00:00,  1.01s/it]

..(dataset: Val) AUC 0.9547201154360819
..(dataset: Test) evaluation



100%|██████████| 56/56 [00:56<00:00,  1.01s/it]


..(dataset: Test) AUC 0.9525032223756267
[Itr 91000] Finished
[Itr 92000] Finished
[Itr 93000] Finished
[Itr 94000] Finished
[Itr 95000] Finished
[Itr 96000] Finished
[Itr 97000] Finished
[Itr 98000] Finished
[Itr 99000] Finished
